# Analyzing Data and Interpreting Images with OpenAI's o1 Reasoning Model vs. GPT

## Introduction
OpenAI's o1 reasoning model is designed for complex problem-solving, data analysis, and image interpretation by simulating a multi-step thought process before generating responses. Unlike traditional GPT models, which produce output in a single pass, reasoning models use internal **reasoning tokens** to explore multiple approaches before finalizing an answer.
<p align="center">
    <img src="https://cdn.openai.com/API/images/guides/reasoning_tokens.png" alt="Reasoning Tokens" width="600">
</p>  

*Source: [OpenAI Reasoning Models Guide](https://platform.openai.com/docs/guides/reasoning)*

**Key Differences: o1 Reasoning Model vs. GPT**
- Multi-step reasoning: o1 evaluates different solutions before selecting the best response.
- Deeper analytical capabilities: Optimized for complex data interpretation tasks.
- Context-aware image analysis: Provides more structured and insightful image descriptions.
- Reasoning Effort Control: Users can adjust the depth of reasoning (`low`, `medium`, `high`).


For more details, refer to the [OpenAI Reasoning Models Guide](https://platform.openai.com/docs/guides/reasoning).


## Purchase and Store API Key

You need to **purchase** your [OpenAI](https://openai.com/) API key and store it securely, such as in **AWS Secrets Manager**.

- **Key Name:** `api_key`  
- **Key Value:** `<your OpenAI API key>`  
- **Secret Name:** `openai`  

## Install Python Libraries

- **openai**: Used to call `o1` and `GPT` models for data analysis and image interpretation.

In [1]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

The following libraries are used in this notebook:

- **boto3**: AWS SDK for Python, used to interact with AWS services.
- **json**: Standard Python library for handling JSON data.
- **IPython.display**: Provides tools to display images, Markdown content, and other rich media in Jupyter Notebook.
- **openai**: Used to call `o1` and `GPT` models for data analysis and image interpretation.
- **pandas**: A powerful library for data manipulation and analysis.
- **pprint**: Pretty prints data structures for better readability.

In [2]:
import boto3
import json
from IPython.display import display, Image, Markdown
from openai import OpenAI
import pandas as pd
from pprint import pprint

## Retrieve API Keys Securely from AWS Secrets Manager

The following function, `get_secret()`, retrieves a secret from **AWS Secrets Manager**. This is a secure way to store and access sensitive credentials, such as API keys, without hardcoding them into the script

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

The following code initializes the OpenAI client using a securely stored API key retrieved from AWS Secrets Manager.

In [4]:
client = OpenAI(api_key= get_secret('openai')['api_key'])

## Load and Analyze the Diamonds Dataset

This notebook uses the **diamonds dataset ([diamonds.csv](https://github.com/lbsocial/data-analysis-with-generative-ai/blob/main/diamonds.csv))**, which contains detailed attributes of diamonds, including weight, color, clarity, and price.

One interesting pattern in the dataset is that **diamonds with "IF" (Internally Flawless) clarity tend to have the lowest average price** compared to other clarity grades. This observation is counterintuitive, as one might expect the highest-clarity diamonds to be the most expensive.

In [5]:
df = pd.read_csv('Department_of_Transportation_Inventory.csv')
data_json = df.to_json(orient="records")
df.head()

,Use Case ID,AI Use Case Name,Agency With AI Use Case,Optional Note Field,Office With AI Use Case,Summary,Current Stage Of Production,Lifecycle Stage Additional Comments,AI Techniques Used,Agency Training Used,...,Included In Enterprise Inventory,Public Data Link,Use Case Code Accessible,Code Included In Agency Inventory,Source Code Public Link,Agency Able To Test Code,Agency Able To Monitor/Audit,Name of Informational System,Withhold Use Case,Explanation To Withhold Use Case
0,3,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,...,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,True,NaN
1,4,Automated Delay detection using voice processing,Department of Transportation,Federal Aviation Administration,ATO,"In order to get a full accounting of delay, au...",In production: less than 1 year,Initial development,Natural Language Processing;,Yes,...,Yes,NaN,Yes,Yes,NaN,Yes,Yes,Wilbur,False,NaN
2,5,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,...,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,True,NaN
3,6,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,...,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,True,NaN
4,7,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,...,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,REDACTED,True,NaN


## Generate Data Analysis Prompt for OpenAI Model

To investigate why diamonds with **IF (Internally Flawless) clarity** have the **lowest average price**, we generate a structured prompt for the OpenAI model. The model will analyze the dataset and generate insights, including **Python code for visualizations**.


In [9]:
data_prompt = f"Analyze the provided data from the Department of Transportation's Inventory of Artificial Intelligence Use Cases. Determine which AI use cases demonstrate the highest potential impact on improving transportation safety, efficiency, or infrastructure modernization. Identify any trends across different DOT agencies and provide Python-generated charts to support your conclusions. Data: {data_json}"
# print(prompt)

## Define a Function to Get Assistance from OpenAI GPT-4o

The following function, `openai_gpt_help()`, sends a prompt to OpenAI's **GPT-4o model** and returns a response. It also prints the number of tokens used in the request.

In [10]:
def openai_gpt_help(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature = 0
    )
    token_usage = response.usage
    
    pprint(f"Tokens used: {token_usage}")

    return response.choices[0].message.content

In [11]:
gpt_result = openai_gpt_help(prompt=data_prompt)

('Tokens used: CompletionUsage(completion_tokens=1126, prompt_tokens=7785, '
 'total_tokens=8911, '
 'completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, '
 'audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), '
 'prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))')


In [12]:
display(Markdown(gpt_result))

To analyze the provided data from the Department of Transportation's Inventory of Artificial Intelligence Use Cases, we will focus on the use cases that are not redacted and assess their potential impact on improving transportation safety, efficiency, or infrastructure modernization. We will also identify trends across different DOT agencies and provide Python-generated charts to support our conclusions.

### Analysis

1. **Use Cases with Potential Impact:**
   - **Automated Delay Detection using Voice Processing (Use Case ID: 4):**
     - **Agency:** Federal Aviation Administration (FAA)
     - **Impact:** This use case aims to improve the efficiency of air traffic management by automating the detection of delays through voice processing. This can lead to more accurate reporting and management of air traffic delays, enhancing overall efficiency.
     - **AI Techniques:** Natural Language Processing (NLP)
     - **Current Stage:** In production for less than 1 year

   - **Surface Report Classifier (SCM/Auto-Class) (Use Case ID: 8):**
     - **Agency:** FAA
     - **Impact:** This use case focuses on classifying surface incident reports, which can significantly improve safety by identifying and categorizing runway incursions and excursions.
     - **AI Techniques:** Support Vector Machines, Gradient Boosting, Neural Networks, NLP
     - **Current Stage:** In production for more than 1 year

   - **Offshore Precipitation Capability (OPC) (Use Case ID: 19):**
     - **Agency:** FAA
     - **Impact:** This use case enhances safety by providing accurate precipitation data, which is crucial for flight safety over oceanic and remote areas.
     - **AI Techniques:** Convolutional Neural Network
     - **Current Stage:** In production for more than 1 year

   - **Machine Learning for Occupant Safety Research (Use Case ID: 27 & 28):**
     - **Agency:** National Highway Traffic Safety Administration (NHTSA)
     - **Impact:** These use cases aim to improve safety by predicting crash parameters and head kinematics, which are critical for understanding and mitigating injury risks in vehicle crashes.
     - **AI Techniques:** Deep Learning Models, Convolutional Neural Networks, Long-Short Term Memory Networks
     - **Current Stage:** Planned (not in production)

2. **Trends Across DOT Agencies:**
   - The FAA is actively using AI to enhance air traffic management and safety, with multiple use cases in production.
   - The NHTSA is focusing on using AI for safety research, particularly in vehicle crash analysis.
   - There is a strong emphasis on using NLP and machine learning techniques across various use cases.

3. **Python-Generated Charts:**

Let's generate some charts to visualize the distribution of AI techniques and the current stage of production for these use cases.

```python
import matplotlib.pyplot as plt
import pandas as pd

# Data for analysis
data = [
    {"Use Case ID": 4, "AI Use Case Name": "Automated Delay detection using voice processing", "Agency": "FAA", "AI Techniques": "NLP", "Current Stage": "In production: less than 1 year"},
    {"Use Case ID": 8, "AI Use Case Name": "Surface Report Classifiier (SCM/Auto-Class)", "Agency": "FAA", "AI Techniques": "SVM, Gradient Boosting, Neural Networks, NLP", "Current Stage": "In production: more than 1 year"},
    {"Use Case ID": 19, "AI Use Case Name": "Offshore Precipitation Capability (OPC)", "Agency": "FAA", "AI Techniques": "Convolutional Neural Network", "Current Stage": "In production: more than 1 year"},
    {"Use Case ID": 27, "AI Use Case Name": "Machine Learning for Occupant Safety Research", "Agency": "NHTSA", "AI Techniques": "Deep Learning, CNN, LSTM", "Current Stage": "Planned (not in production)"},
    {"Use Case ID": 28, "AI Use Case Name": "Machine Learning for Occupant Safety Research", "Agency": "NHTSA", "AI Techniques": "Deep Learning, CNN", "Current Stage": "Planned (not in production)"}
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Plotting AI Techniques
plt.figure(figsize=(10, 6))
df['AI Techniques'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribution of AI Techniques')
plt.xlabel('AI Techniques')
plt.ylabel('Number of Use Cases')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Plotting Current Stage of Production
plt.figure(figsize=(10, 6))
df['Current Stage'].value_counts().plot(kind='bar', color='lightgreen')
plt.title('Current Stage of Production')
plt.xlabel('Stage')
plt.ylabel('Number of Use Cases')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
```

### Conclusion

The analysis shows that the FAA is leading in implementing AI use cases that are already in production, focusing on improving air traffic management and safety. The NHTSA is exploring AI for vehicle safety research, which is still in the planning stages. The use of NLP and machine learning techniques is prevalent across these use cases, indicating a trend towards leveraging these technologies for enhancing transportation safety and efficiency.

## Define a Function to Get Assistance from OpenAI o1 Model  

The following function, `openai_o_help()`, sends a prompt to OpenAI's **o1 reasoning model** and returns a response.  

### Key Differences Between o1 and GPT Models:
- **Reasoning Effort**: The o1 model allows users to control reasoning depth using `reasoning_effort` (`low`, `medium`, `high`).  
- **No Temperature Parameter**: Unlike GPT models, **o1 does not support `temperature`**.  
- **Developer Messages Replace System Messages**:  
  - Starting with `o1-2024-12-17`, **developer messages** replace **system messages** to align with chain-of-command behavior.  

### Best Practices for Prompting o1  
- **Keep prompts simple and direct.**  
- **Avoid chain-of-thought prompts.** o1 reasons internally, so step-by-step instructions aren't needed.  
- **Use delimiters for clarity.** Use Markdown, XML tags, or section titles.  
- **Try zero-shot first.** If needed, add few-shot examples that closely match your goal.  
- **Be explicit.** Clearly define success criteria and constraints.  
- **Markdown is disabled by default.** To enable, start with `"Formatting re-enabled"`.  

Source: [OpenAI Reasoning Models Best Practices Guide](https://platform.openai.com/docs/guides/reasoning-best-practices).  


In [13]:
def openai_o_help(prompt):
    messages = [ {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model='o1',
        reasoning_effort="high", # low, medium or high
        messages=messages,

    )
    token_usage = response.usage
    
    pprint(f"Tokens used: {token_usage}")

    return response.choices[0].message.content

In [14]:
o1_result = openai_o_help(prompt=data_prompt)

('Tokens used: CompletionUsage(completion_tokens=6263, prompt_tokens=7784, '
 'total_tokens=14047, '
 'completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, '
 'audio_tokens=0, reasoning_tokens=4160, rejected_prediction_tokens=0), '
 'prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))')


In [15]:
print(o1_result)

Below is an analysis of the publicly listed AI use cases (i.e., those not marked “Withhold Use Case”) from the Department of Transportation’s Inventory of Artificial Intelligence Use Cases, highlighting which appear to have the greatest potential to improve safety, efficiency, or infrastructure modernization. Following the analysis, you will find example Python code snippets (using pandas and matplotlib) that illustrate how one might generate basic charts to support these conclusions.

────────────────────────────────────────────────────────
1) Identifying High-Impact AI Use Cases
────────────────────────────────────────────────────────

Note: Only use cases where "Withhold Use Case" = false are analyzed below. Many entries in the dataset are fully or partially redacted and marked to be withheld.

Below is a short summary of the non-withheld use cases, grouped by their primary potential impact areas (safety, efficiency, and/or infrastructure modernization). Several use cases provide ov

1. How the model's reasoning supported your analysis: The model filtered the dataset to include only non-redacted use cases, allowing a focused analysis on transparent, operational AI applications. By identifying which agencies have the most visible AI deployments, the model used frequency analysis and data visualization to highlight where AI is actively improving transportation outcomes. This pattern supports the conclusion that certain agencies are more mature in their use of AI to enhance real-time responsiveness, a key factor in public service efficiency and safety.

2. Whether this approach could be applied to real-world intelligence workflows: This approach closely mirrors workflows used in open-source intelligence (OSINT) and applied AI threat analysis. The process of filtering out redacted or classified content aligns with standard intelligence procedures for identifying actionable insights from open datasets. Analyzing trends such as counting the number of AI deployments by agency—helps prioritize resource allocation and supports risk assessments in national security contexts. Additionally, the use of Python-generated charts enhances the clarity of findings, enabling analysts to communicate trends effectively to policymakers and decision-makers. This analytical method could easily be repurposed to monitor AI developments in adversarial nations, evaluate emerging cyber threat landscapes, or assess the domestic policy impacts of new technologies.

3. Any limitations or ethical concerns you encountered: Several limitations and ethical concerns emerged in the analysis. Much of the dataset was redacted, limiting insight into AI use and strategy. Some use cases lacked detail, making it hard to assess performance or transparency. Ethically, the absence of oversight indicators and the use of AI in surveillance raise concerns about privacy and accountability. While withholding data may protect security, it also reduces public trust. These issues highlight the need for transparency and ethical oversight in government AI use.

## References  
- **OpenAI Reasoning Models Guide**: [OpenAI](https://platform.openai.com/docs/guides/reasoning)  
- **OpenAI Reasoning Models Best Practices Guide**: [OpenAI](https://platform.openai.com/docs/guides/reasoning-best-practices)  
- **Colin Jarvis. “Reasoning with O1.” DeepLearning.AI.** Accessed February 14, 2025. [DeepLearning.AI](https://www.deeplearning.ai/short-courses/reasoning-with-o1/)  